# BlackBelt Consulting

## 1. Benson Project

### 1.1 Data Collection

In [1]:
import pandas as pd
import datetime

In [16]:
numdays = 21  # just setting the number of days we want to take back 1 week from 06-29
# we can increase it later, this is just so we can start with a "small" data set

In [17]:
date_time_str = '2019-06-22 08:15:27.243860'  # penultimate date of the range in the MTA website
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')  
# transforming it to a date_time object
date_time_obj

datetime.datetime(2019, 6, 22, 8, 15, 27, 243860)

In [18]:
date_list = [date_time_obj - datetime.timedelta(days=x) for x in range(0, numdays, 7)]
# creating a list of dates that starts on the penultimate date and goes back the numdays we've set 
# (jumping 7 days each time)
date_list

[datetime.datetime(2019, 6, 22, 8, 15, 27, 243860),
 datetime.datetime(2019, 6, 15, 8, 15, 27, 243860),
 datetime.datetime(2019, 6, 8, 8, 15, 27, 243860)]

In [19]:
# transforming the dates into strings and putting in a list:

url_dates = []

for i in date_list:
    year = str(i.year).replace("20", "")
    day = str(i.day)
    if i.day < 10:
        day = "0" + day
    month = str(i.month)
    if i.month < 10:
        month = "0" + month
    date_str = year + month + day
    url_dates.append(date_str)

url_dates

['190622', '190615', '190608']

In [20]:
# starting the dataframe with the last available date, which is june 29, 19:
df = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190629.txt")
df.head()

C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  00:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  04:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  08:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  12:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  16:00:00   

      DESC  ENTRIES  \
0  REGULAR  7107725   
1  REGULAR  7107738   
2  REGULAR  7107761   
3  REGULAR  7107858   
4  REGULAR  7108075   

   EXITS                                                                 
0                                            2407457                     
1                                            2407465                     
2                                            2407491                     
3                                            2407541                     
4                                            2407581

In [21]:
# concatenating each new date to the original dataframe:
for url in url_dates:
    df2 = pd.read_csv(
        "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt".format(url))
    df = pd.concat([df, df2], ignore_index=True)

In [23]:
df.head()

C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  00:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  04:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  08:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  12:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  16:00:00   

      DESC  ENTRIES  \
0  REGULAR  7107725   
1  REGULAR  7107738   
2  REGULAR  7107761   
3  REGULAR  7107858   
4  REGULAR  7108075   

   EXITS                                                                 
0                                            2407457                     
1                                            2407465                     
2                                            2407491                     
3                                            2407541                     
4                                            2407581

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823385 entries, 0 to 823384
Data columns (total 11 columns):
C/A                                                                     823385 non-null object
UNIT                                                                    823385 non-null object
SCP                                                                     823385 non-null object
STATION                                                                 823385 non-null object
LINENAME                                                                823385 non-null object
DIVISION                                                                823385 non-null object
DATE                                                                    823385 non-null object
TIME                                                                    823385 non-null object
DESC                                                                    823385 non-null object
ENTRIES                           

In [25]:
df.columns = [column.strip() for column in df.columns]

In [26]:
df["DATE_TIME"] = pd.to_datetime(df["DATE"] + " " + df["TIME"])

In [29]:
# Data from - Jun 2019
df.DATE.value_counts().sort_index()

06/01/2019    29066
06/02/2019    28968
06/03/2019    30022
06/04/2019    29513
06/05/2019    29484
06/06/2019    29213
06/07/2019    28745
06/08/2019    29030
06/09/2019    29133
06/10/2019    29351
06/11/2019    29113
06/12/2019    30023
06/13/2019    29276
06/14/2019    29008
06/15/2019    29605
06/16/2019    29322
06/17/2019    29421
06/18/2019    29599
06/19/2019    29614
06/20/2019    29556
06/21/2019    30360
06/22/2019    29217
06/23/2019    29170
06/24/2019    29479
06/25/2019    29437
06/26/2019    29417
06/27/2019    29464
06/28/2019    29779
Name: DATE, dtype: int64

In [33]:
df["TURNSTILE_ID"] = df["C/A"] + " " + df["UNIT"] + " " + df["SCP"] + " " + df["STATION"]

In [34]:
df.head()

C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  00:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  04:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  08:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  12:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/22/2019  16:00:00   

      DESC  ENTRIES    EXITS           DATE_TIME              TURNSTILE_ID  
0  REGULAR  7107725  2407457 2019-06-22 00:00:00  A002 R051 02-00-00 59 ST  
1  REGULAR  7107738  2407465 2019-06-22 04:00:00  A002 R051 02-00-00 59 ST  
2  REGULAR  7107761  2407491 2019-06-22 08:00:00  A002 R051 02-00-00 59 ST  
3  REGULAR  7107858  2407541 2019-06-22 12:00:00  A002 R051 02-00-00 59 ST  
4  REGULAR  7108075  2407581 2019-06-22 16:00:00  A002 R051 02-00-00 59 ST

In [46]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(df
 .groupby(["TURNSTILE_ID", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(30)

TURNSTILE_ID           DATE_TIME  ENTRIES
583293          R174 R034 00-00-03 125 ST 2019-06-08 00:00:00        2
194099    N045 R187 01-00-02 81 ST-MUSEUM 2019-06-01 17:00:00        2
193942    N045 R187 01-00-01 81 ST-MUSEUM 2019-06-02 17:00:00        2
194324    N045 R187 01-06-00 81 ST-MUSEUM 2019-06-11 21:00:00        2
660217           R249 R179 01-05-00 86 ST 2019-06-27 00:00:00        2
679477  R290 R161 00-00-00 KINGSBRIDGE RD 2019-06-07 05:00:00        2
660221           R249 R179 01-05-00 86 ST 2019-06-27 04:00:00        2
583125          R174 R034 00-00-02 125 ST 2019-06-08 00:00:00        2
193937    N045 R187 01-00-01 81 ST-MUSEUM 2019-06-01 17:00:00        2
194104    N045 R187 01-00-02 81 ST-MUSEUM 2019-06-02 17:00:00        2
660228           R249 R179 01-05-00 86 ST 2019-06-27 12:00:00        2
194269    N045 R187 01-06-00 81 ST-MUSEUM 2019-06-02 17:00:00        2
660225           R249 R179 01-05-00 86 ST 2019-06-27 08:00:00        2
582957          R174 R034 00-00-01 125 ST 2019-06-08 00:00:00        2
193777    N045 R187 01-00-00 81 ST-MUSEUM 2019-06-02 17:00:00        2
627395           R229 R143 01-00-00 28 ST 2019-06-27 17:00:00        2
193772    N045 R187 01-00-00 81 ST-MUSEUM 2019-06-01 17:00:00        2
194193    N045 R187 01-00-02 81 ST-MUSEUM 2019-06-17 09:00:00        2
548927  R139 R031 04-00-02 34 ST-PENN STA 2019-06-02 08:00:00        1
548919  R139 R031 04-00-02 34 ST-PENN STA 2019-06-01 00:00:00        1
548920  R139 R031 04-00-02 34 ST-PENN STA 2019-06-01 04:00:00        1
548921  R139 R031 04-00-02 34 ST-PENN STA 2019-06-01 08:00:00        1
548922  R139 R031 04-00-02 34 ST-PENN STA 2019-06-01 12:00:00        1
548923  R139 R031 04-00-02 34 ST-PENN STA 2019-06-01 16:00:00        1
548924  R139 R031 04-00-02 34 ST-PENN STA 2019-06-01 20:00:00        1
548925  R139 R031 04-00-02 34 ST-PENN STA 2019-06-02 00:00:00        1
548926  R139 R031 04-00-02 34 ST-PENN STA 2019-06-02 04:00:00        1
548928  R139 R031 04-00-02 34 ST-PENN STA 2019-06-02 12:00:00        1
548918  R139 R031 04-00-01 34 ST-PENN STA 2019-06-28 20:00:00        1
548896  R139 R031 04-00-01 34 ST-PENN STA 2019-06-25 04:00:00        1

In [47]:
# On some days, we seem to have two entries for same time.  Let's take a look
mask = ((df["TURNSTILE_ID"] == "R174 R034 00-00-03 125 ST") &
(df["DATE_TIME"].dt.date == datetime.datetime(2019, 6, 8).date()))

df[mask]

C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
558895  R174  R034  00-00-03  125 ST        1      IRT  06/08/2019  00:00:00   
558896  R174  R034  00-00-03  125 ST        1      IRT  06/08/2019  00:00:00   
558897  R174  R034  00-00-03  125 ST        1      IRT  06/08/2019  08:00:00   
558898  R174  R034  00-00-03  125 ST        1      IRT  06/08/2019  12:00:00   
558899  R174  R034  00-00-03  125 ST        1      IRT  06/08/2019  16:00:00   
558900  R174  R034  00-00-03  125 ST        1      IRT  06/08/2019  20:00:00   

              DESC  ENTRIES    EXITS           DATE_TIME  \
558895     REGULAR  3828801  2853462 2019-06-08 00:00:00   
558896  RECOVR AUD  3828804  2853463 2019-06-08 00:00:00   
558897     REGULAR  3828931  2853610 2019-06-08 08:00:00   
558898     REGULAR  3829164  2853787 2019-06-08 12:00:00   
558899     REGULAR  3829514  2854060 2019-06-08 16:00:00   
558900     REGULAR  3829854  2854357 2019-06-08 20:00:00   

                     TURNSTILE_ID  
558895  R174 R034 00-00-03 125 ST  
558896  R174 R034 00-00-03 125 ST  
558897  R174 R034 00-00-03 125 ST  
558898  R174 R034 00-00-03 125 ST  
558899  R174 R034 00-00-03 125 ST  
558900  R174 R034 00-00-03 125 ST

In [48]:
# On some days, we seem to have two entries for same time.  Let's take a look
mask = ((df["TURNSTILE_ID"] == "N045 R187 01-00-02 81 ST-MUSEUM") &
(df["DATE_TIME"].dt.date == datetime.datetime(2019, 6, 1).date()))

df[mask]

C/A  UNIT       SCP       STATION LINENAME DIVISION        DATE  \
666700  N045  R187  01-00-02  81 ST-MUSEUM       BC      IND  06/01/2019   
666701  N045  R187  01-00-02  81 ST-MUSEUM       BC      IND  06/01/2019   
666702  N045  R187  01-00-02  81 ST-MUSEUM       BC      IND  06/01/2019   
666703  N045  R187  01-00-02  81 ST-MUSEUM       BC      IND  06/01/2019   
666704  N045  R187  01-00-02  81 ST-MUSEUM       BC      IND  06/01/2019   
666705  N045  R187  01-00-02  81 ST-MUSEUM       BC      IND  06/01/2019   
666706  N045  R187  01-00-02  81 ST-MUSEUM       BC      IND  06/01/2019   

            TIME        DESC  ENTRIES   EXITS           DATE_TIME  \
666700  01:00:00     REGULAR  4323386  989690 2019-06-01 01:00:00   
666701  05:00:00     REGULAR  4323388  989692 2019-06-01 05:00:00   
666702  09:00:00     REGULAR  4323394  989696 2019-06-01 09:00:00   
666703  13:00:00     REGULAR  4323510  989797 2019-06-01 13:00:00   
666704  17:00:00     REGULAR  4324002  989878 2019-06-01 17:00:00   
666705  17:00:00  RECOVR AUD  4323998  989878 2019-06-01 17:00:00   
666706  21:00:00     REGULAR  4324268  989897 2019-06-01 21:00:00   

                           TURNSTILE_ID  
666700  N045 R187 01-00-02 81 ST-MUSEUM  
666701  N045 R187 01-00-02 81 ST-MUSEUM  
666702  N045 R187 01-00-02 81 ST-MUSEUM  
666703  N045 R187 01-00-02 81 ST-MUSEUM  
666704  N045 R187 01-00-02 81 ST-MUSEUM  
666705  N045 R187 01-00-02 81 ST-MUSEUM  
666706  N045 R187 01-00-02 81 ST-MUSEUM

In [49]:
# Looking over more examples, these duplicates don't seem to show significant differences. 
# Therefore, we will get rid of the duplicate entries:
df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [50]:
# No problems anymore:
(df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(15)

C/A  UNIT       SCP         STATION           DATE_TIME  ENTRIES
0       A002  R051  02-00-00           59 ST 2019-06-01 00:00:00        1
548916  R139  R031  04-00-01  34 ST-PENN STA 2019-06-28 12:00:00        1
548906  R139  R031  04-00-01  34 ST-PENN STA 2019-06-26 20:00:00        1
548907  R139  R031  04-00-01  34 ST-PENN STA 2019-06-27 00:00:00        1
548908  R139  R031  04-00-01  34 ST-PENN STA 2019-06-27 04:00:00        1
548909  R139  R031  04-00-01  34 ST-PENN STA 2019-06-27 08:00:00        1
548910  R139  R031  04-00-01  34 ST-PENN STA 2019-06-27 12:00:00        1
548911  R139  R031  04-00-01  34 ST-PENN STA 2019-06-27 16:00:00        1
548912  R139  R031  04-00-01  34 ST-PENN STA 2019-06-27 20:00:00        1
548913  R139  R031  04-00-01  34 ST-PENN STA 2019-06-28 00:00:00        1
548914  R139  R031  04-00-01  34 ST-PENN STA 2019-06-28 04:00:00        1
548915  R139  R031  04-00-01  34 ST-PENN STA 2019-06-28 08:00:00        1
548917  R139  R031  04-00-01  34 ST-PENN STA 2019-06-28 16:00:00        1
548879  R139  R031  04-00-01  34 ST-PENN STA 2019-06-22 08:00:00        1
548918  R139  R031  04-00-01  34 ST-PENN STA 2019-06-28 20:00:00        1

For our data we are focusing only in the entries since we believe this is a better approach to gauge the number of people focusing on the ads.

In [51]:
# Drop Exits and Desc Column.  To prevent errors in multiple run of cell, errors on drop is ignored
df = df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

In [68]:
df.reset_index(drop=True)

C/A  UNIT       SCP        STATION LINENAME DIVISION        DATE  \
0       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
1       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
2       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
3       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
4       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
5       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
6       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/27/2019   
7       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/27/2019   
8       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/27/2019   
9       TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/27/2019   
10      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/27/2019   
11      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/27/2019   
12      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/26/2019   
13      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/26/2019   
14      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/26/2019   
15      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/26/2019   
16      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/26/2019   
17      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/26/2019   
18      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/25/2019   
19      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/25/2019   
20      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/25/2019   
21      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/25/2019   
22      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/25/2019   
23      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/25/2019   
24      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/24/2019   
25      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/24/2019   
26      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/24/2019   
27      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/24/2019   
28      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/24/2019   
29      TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/24/2019   
...       ...   ...       ...            ...      ...      ...         ...   
823337   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/05/2019   
823338   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/05/2019   
823339   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/05/2019   
823340   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/05/2019   
823341   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/05/2019   
823342   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/05/2019   
823343   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/04/2019   
823344   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/04/2019   
823345   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/04/2019   
823346   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/04/2019   
823347   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/04/2019   
823348   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/04/2019   
823349   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/03/2019   
823350   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/03/2019   
823351   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/03/2019   
823352   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/03/2019   
823353   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/03/2019   
823354   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/03/2019   
823355   A002  R051  02-00-00          59 ST  NQR456W      BMT  06/02/2019   
823356   A002  R051  02-00-00   

In [70]:
# There seems to be a problem with 23rd street (and others) because different stations are named the same way:
df[df["STATION"] == "23 ST"].groupby(["LINENAME"]).count()

C/A  UNIT   SCP  STATION  DIVISION  DATE  TIME  ENTRIES  DATE_TIME  \
LINENAME                                                                        
1         1184  1184  1184     1184      1184  1184  1184     1184       1184   
6         4277  4277  4277     4277      4277  4277  4277     4277       4277   
CE        1878  1878  1878     1878      1878  1878  1878     1878       1878   
FM        1724  1724  1724     1724      1724  1724  1724     1724       1724   
NRW       1769  1769  1769     1769      1769  1769  1769     1769       1769   

          TURNSTILE_ID  
LINENAME                
1                 1184  
6                 4277  
CE                1878  
FM                1724  
NRW               1769

In [71]:
# Let's create a new column that concatenates the station and the lines to differenciate them:
df["STATION_LINE"] = df["STATION"] + " " + df["LINENAME"]

C/A  UNIT       SCP        STATION LINENAME DIVISION        DATE  \
205962  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
205961  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
205960  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
205959  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   
205958  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  06/28/2019   

            TIME  ENTRIES           DATE_TIME  \
205962  21:00:00     5554 2019-06-28 21:00:00   
205961  17:00:00     5554 2019-06-28 17:00:00   
205960  13:00:00     5554 2019-06-28 13:00:00   
205959  09:00:00     5554 2019-06-28 09:00:00   
205958  05:00:00     5554 2019-06-28 05:00:00   

                             TURNSTILE_ID     STATION_LINE  
205962  TRAM2 R469 00-05-01 RIT-ROOSEVELT  RIT-ROOSEVELT R  
205961  TRAM2 R469 00-05-01 RIT-ROOSEVELT  RIT-ROOSEVELT R  
205960  TRAM2 R469 00-05-01 RIT-ROOSEVELT  RIT-ROOSEVELT R  
205959  TRAM2 R469 00-05-01 RIT-ROOSEVELT  RIT-ROOSEVELT R  
205958  TRAM2 R469 00-05-01 RIT-ROOSEVELT  RIT-ROOSEVELT R

In [94]:
turnstiles_daily = (df
                        .groupby(["C/A", "UNIT", "SCP", "STATION","TURNSTILE_ID", "LINENAME", "DATE"],as_index=False)
                        .ENTRIES.max())

In [95]:
turnstiles_daily

C/A  UNIT       SCP        STATION  \
0        A002  R051  02-00-00          59 ST   
1        A002  R051  02-00-00          59 ST   
2        A002  R051  02-00-00          59 ST   
3        A002  R051  02-00-00          59 ST   
4        A002  R051  02-00-00          59 ST   
5        A002  R051  02-00-00          59 ST   
6        A002  R051  02-00-00          59 ST   
7        A002  R051  02-00-00          59 ST   
8        A002  R051  02-00-00          59 ST   
9        A002  R051  02-00-00          59 ST   
10       A002  R051  02-00-00          59 ST   
11       A002  R051  02-00-00          59 ST   
12       A002  R051  02-00-00          59 ST   
13       A002  R051  02-00-00          59 ST   
14       A002  R051  02-00-00          59 ST   
15       A002  R051  02-00-00          59 ST   
16       A002  R051  02-00-00          59 ST   
17       A002  R051  02-00-00          59 ST   
18       A002  R051  02-00-00          59 ST   
19       A002  R051  02-00-00          59 ST   
20       A002  R051  02-00-00          59 ST   
21       A002  R051  02-00-00          59 ST   
22       A002  R051  02-00-00          59 ST   
23       A002  R051  02-00-00          59 ST   
24       A002  R051  02-00-00          59 ST   
25       A002  R051  02-00-00          59 ST   
26       A002  R051  02-00-00          59 ST   
27       A002  R051  02-00-00          59 ST   
28       A002  R051  02-00-01          59 ST   
29       A002  R051  02-00-01          59 ST   
...       ...   ...       ...            ...   
136122  TRAM2  R469  00-05-00  RIT-ROOSEVELT   
136123  TRAM2  R469  00-05-00  RIT-ROOSEVELT   
136124  TRAM2  R469  00-05-00  RIT-ROOSEVELT   
136125  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136126  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136127  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136128  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136129  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136130  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136131  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136132  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136133  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136134  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136135  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136136  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136137  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136138  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136139  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136140  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136141  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136142  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136143  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136144  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136145  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136146  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136147  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136148  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136149  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136150  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136151  TRAM2  R469  00-05-01  RIT-ROOSEVELT   

                             TURNSTILE_ID LINENAME        DATE  ENTRIES  
0                A002 R051 02-00-00 59 ST  NQR456W  06/01/2019  7080817  
1                A002 R051 02-00-00 59 ST  NQR456W  06/02/2019  7081567  
2                A002 R051 02-00-00 59 ST  NQR456W  06/03/2019  7082949  
3                A002 R051 02-00-00 59 ST  NQR456W  06/04/2019  7084565  
4                A002 R051 02-00-00 59 ST  NQR456W  06/05/2019  7086109  
5                A002 R051 02-00-00 59 ST  NQR456W  06/06/2019  7087657  
6                A002 R051 02-00-00 59 ST  NQR456W  06/07/2019  7089268  
7                A002 R051 02-00-00 59 ST  NQR456W  06/08/2019  7090115  
8                A002 R051 02-00-00 59 ST  NQR456W  06/09/2019  7090874  
9                A002 R051 02-00-00 59 ST  NQR456W  06/10/2019  7092269  
10               A002 R051 02-00-00 59 ST  NQR456W  06/11/2019  7093803  
11               A002 R051 02-00-00 59 ST  NQR456W  06/12/2019  7095407  
12               A002 R051 02-00-00 59 ST  NQR456W  06/13/2019  7096917

In [96]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                       .apply(lambda grp: grp.shift(1)))  

In [97]:
turnstiles_daily

C/A  UNIT       SCP        STATION  \
0        A002  R051  02-00-00          59 ST   
1        A002  R051  02-00-00          59 ST   
2        A002  R051  02-00-00          59 ST   
3        A002  R051  02-00-00          59 ST   
4        A002  R051  02-00-00          59 ST   
5        A002  R051  02-00-00          59 ST   
6        A002  R051  02-00-00          59 ST   
7        A002  R051  02-00-00          59 ST   
8        A002  R051  02-00-00          59 ST   
9        A002  R051  02-00-00          59 ST   
10       A002  R051  02-00-00          59 ST   
11       A002  R051  02-00-00          59 ST   
12       A002  R051  02-00-00          59 ST   
13       A002  R051  02-00-00          59 ST   
14       A002  R051  02-00-00          59 ST   
15       A002  R051  02-00-00          59 ST   
16       A002  R051  02-00-00          59 ST   
17       A002  R051  02-00-00          59 ST   
18       A002  R051  02-00-00          59 ST   
19       A002  R051  02-00-00          59 ST   
20       A002  R051  02-00-00          59 ST   
21       A002  R051  02-00-00          59 ST   
22       A002  R051  02-00-00          59 ST   
23       A002  R051  02-00-00          59 ST   
24       A002  R051  02-00-00          59 ST   
25       A002  R051  02-00-00          59 ST   
26       A002  R051  02-00-00          59 ST   
27       A002  R051  02-00-00          59 ST   
28       A002  R051  02-00-01          59 ST   
29       A002  R051  02-00-01          59 ST   
...       ...   ...       ...            ...   
136122  TRAM2  R469  00-05-00  RIT-ROOSEVELT   
136123  TRAM2  R469  00-05-00  RIT-ROOSEVELT   
136124  TRAM2  R469  00-05-00  RIT-ROOSEVELT   
136125  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136126  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136127  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136128  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136129  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136130  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136131  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136132  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136133  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136134  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136135  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136136  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136137  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136138  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136139  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136140  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136141  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136142  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136143  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136144  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136145  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136146  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136147  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136148  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136149  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136150  TRAM2  R469  00-05-01  RIT-ROOSEVELT   
136151  TRAM2  R469  00-05-01  RIT-ROOSEVELT   

                             TURNSTILE_ID LINENAME        DATE  ENTRIES  \
0                A002 R051 02-00-00 59 ST  NQR456W  06/01/2019  7080817   
1                A002 R051 02-00-00 59 ST  NQR456W  06/02/2019  7081567   
2                A002 R051 02-00-00 59 ST  NQR456W  06/03/2019  7082949   
3                A002 R051 02-00-00 59 ST  NQR456W  06/04/2019  7084565   
4                A002 R051 02-00-00 59 ST  NQR456W  06/05/2019  7086109   
5                A002 R051 02-00-00 59 ST  NQR456W  06/06/2019  7087657   
6                A002 R051 02-00-00 59 ST  NQR456W  06/07/2019  7089268   
7                A002 R051 02-00-00 59 ST  NQR456W  06/08/2019  7090115   
8                A002 R051 02-00-00 59 ST  NQR456W  06/09/2019  7090874   
9                A002 R051 02-00-00 59 ST  NQR456W  06/10/2019  7092269   
10               A002 R051 02-00-00 59 ST  NQR456W  06/11/2019  7093803   
11               A002 R051 02-00-00 59 ST  NQR456W  06/12/2019  7095407   
12               A002 R051 02-00-00 59 ST  NQR456W  06/13/

In [98]:
# Drop the rows for the earliest date in the df
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [99]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

4407 3125272.0
1376283 8703.0
2801 4317416.0
1790 13967417.0
221 7246254.0
4050923 5903657.0
7246468 4050451.0
524600 5853002.0
595060 57022793.0
5573286 104608160.0
459396 4526857.0
458752 117849318.0
228 1393341.0
286 3466273.0
1828704206 9687851.0
342 4086581.0
1834 2588198.0
2624 4676995.0
1963 621485465.0
710 1253355.0
1875 1652257.0
3642 1524744.0
1447 1048102.0
2398 1252507.0
1773 1573067.0
2749 3127400.0
4093 1947791.0
4105 1216440.0
1877 1310209.0
3488 1063837.0
3493 2409981.0
3450 1093598.0
2957 1461430.0
4608 1055224.0
3003 2725550.0
4220 2041814.0
1 1910177.0
1185 1003916.0
1152 3210019.0
5350 1395727.0
4407 2286257.0
4435 1696384.0
3984 1058923.0
0 1078800.0
4661 2793550.0
659 1327071.0
1338 1427069.0
9513 1560406.0
2966 2029275.0
452993419 1830432.0
881626829 1942743949.0
88840996 1258666495.0
704681758 2758310.0
868286442 101740072.0
360 1488298.0


In [107]:
turnstiles_daily["DAILY_ENTRIES"].sort_values(ascending=False)

79921     974322.0
42108     966219.0
78698     965785.0
77582     935445.0
81877     902748.0
76824     883173.0
79735     866397.0
42109     851681.0
79595     847588.0
77498     794981.0
81753     731850.0
76020     728692.0
79819     618465.0
77943     605457.0
20143     595060.0
77915     563375.0
78894     556482.0
84198     555330.0
82122     540700.0
82652     534728.0
78782     532919.0
78390     525927.0
17093     524600.0
42081     496176.0
4508      489304.0
36822     459396.0
37871     458752.0
75737     449761.0
76880     431993.0
78083     430706.0
            ...   
29036          0.0
29038          0.0
29045          0.0
29047          0.0
29051          0.0
29053          0.0
29054          0.0
29056          0.0
28977          0.0
28975          0.0
28955          0.0
28974          0.0
28957          0.0
28958          0.0
28959          0.0
28960          0.0
28961          0.0
28962          0.0
28963          0.0
28964          0.0
28965          0.0
28966       

In [123]:
turnstiles_daily[turnstiles_daily["STATION"] == "JOURNAL SQUARE"]

C/A  UNIT       SCP         STATION  \
76706  PTH03  R552  00-00-00  JOURNAL SQUARE   
76707  PTH03  R552  00-00-00  JOURNAL SQUARE   
76708  PTH03  R552  00-00-00  JOURNAL SQUARE   
76709  PTH03  R552  00-00-00  JOURNAL SQUARE   
76710  PTH03  R552  00-00-00  JOURNAL SQUARE   
76711  PTH03  R552  00-00-00  JOURNAL SQUARE   
76712  PTH03  R552  00-00-00  JOURNAL SQUARE   
76713  PTH03  R552  00-00-00  JOURNAL SQUARE   
76714  PTH03  R552  00-00-00  JOURNAL SQUARE   
76715  PTH03  R552  00-00-00  JOURNAL SQUARE   
76716  PTH03  R552  00-00-00  JOURNAL SQUARE   
76717  PTH03  R552  00-00-00  JOURNAL SQUARE   
76718  PTH03  R552  00-00-00  JOURNAL SQUARE   
76719  PTH03  R552  00-00-00  JOURNAL SQUARE   
76720  PTH03  R552  00-00-00  JOURNAL SQUARE   
76721  PTH03  R552  00-00-00  JOURNAL SQUARE   
76722  PTH03  R552  00-00-00  JOURNAL SQUARE   
76723  PTH03  R552  00-00-00  JOURNAL SQUARE   
76724  PTH03  R552  00-00-00  JOURNAL SQUARE   
76725  PTH03  R552  00-00-00  JOURNAL SQUARE   
76726  PTH03  R552  00-00-00  JOURNAL SQUARE   
76727  PTH03  R552  00-00-00  JOURNAL SQUARE   
76728  PTH03  R552  00-00-00  JOURNAL SQUARE   
76729  PTH03  R552  00-00-00  JOURNAL SQUARE   
76730  PTH03  R552  00-00-00  JOURNAL SQUARE   
76731  PTH03  R552  00-00-00  JOURNAL SQUARE   
76732  PTH03  R552  00-00-00  JOURNAL SQUARE   
76734  PTH03  R552  00-00-01  JOURNAL SQUARE   
76735  PTH03  R552  00-00-01  JOURNAL SQUARE   
76736  PTH03  R552  00-00-01  JOURNAL SQUARE   
...      ...   ...       ...             ...   
77289  PTH03  R552  00-02-00  JOURNAL SQUARE   
77290  PTH03  R552  00-02-00  JOURNAL SQUARE   
77291  PTH03  R552  00-02-00  JOURNAL SQUARE   
77293  PTH03  R552  00-02-01  JOURNAL SQUARE   
77294  PTH03  R552  00-02-01  JOURNAL SQUARE   
77295  PTH03  R552  00-02-01  JOURNAL SQUARE   
77296  PTH03  R552  00-02-01  JOURNAL SQUARE   
77297  PTH03  R552  00-02-01  JOURNAL SQUARE   
77298  PTH03  R552  00-02-01  JOURNAL SQUARE   
77299  PTH03  R552  00-02-01  JOURNAL SQUARE   
77300  PTH03  R552  00-02-01  JOURNAL SQUARE   
77301  PTH03  R552  00-02-01  JOURNAL SQUARE   
77302  PTH03  R552  00-02-01  JOURNAL SQUARE   
77303  PTH03  R552  00-02-01  JOURNAL SQUARE   
77304  PTH03  R552  00-02-01  JOURNAL SQUARE   
77305  PTH03  R552  00-02-01  JOURNAL SQUARE   
77306  PTH03  R552  00-02-01  JOURNAL SQUARE   
77307  PTH03  R552  00-02-01  JOURNAL SQUARE   
77308  PTH03  R552  00-02-01  JOURNAL SQUARE   
77309  PTH03  R552  00-02-01  JOURNAL SQUARE   
77310  PTH03  R552  00-02-01  JOURNAL SQUARE   
77311  PTH03  R552  00-02-01  JOURNAL SQUARE   
77312  PTH03  R552  00-02-01  JOURNAL SQUARE   
77313  PTH03  R552  00-02-01  JOURNAL SQUARE   
77314  PTH03  R552  00-02-01  JOURNAL SQUARE   
77315  PTH03  R552  00-02-01  JOURNAL SQUARE   
77316  PTH03  R552  00-02-01  JOURNAL SQUARE   
77317  PTH03  R552  00-02-01  JOURNAL SQUARE   
77318  PTH03  R552  00-02-01  JOURNAL SQUARE   
77319  PTH03  R552  00-02-01  JOURNAL SQUARE   

                             TURNSTILE_ID LINENAME        DATE  ENTRIES  \
76706  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/02/2019    82605   
76707  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/03/2019    83041   
76708  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/04/2019    83436   
76709  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/05/2019    83881   
76710  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/06/2019    84312   
76711  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/07/2019    84346   
76712  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/08/2019      341   
76713  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/09/2019      479   
76714  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/10/2019      924   
76715  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/11/2019     1404   
76716  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/12/2019     1861   
76717  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/13/2019     2356   
76718  PTH03 R552 00-00-00 JOURNAL SQUARE        1  06/14

In [108]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()  # automatic seaborn settings

%matplotlib inline

In [109]:
sns.distplot(turnstiles_daily['DAILY_ENTRIES']
             [turnstiles_daily['DAILY_ENTRIES'] < 50000]);

In [122]:
# first get daily entries by station
stations_daily = \
    (turnstiles_daily.groupby(['STATION','DATE'])['DAILY_ENTRIES'].sum()
                 .reset_index())  

stations_daily[stations_daily["STATION"] == "JOURNAL SQUARE"]

STATION        DATE  DAILY_ENTRIES
6947  JOURNAL SQUARE  06/02/2019        11134.0
6948  JOURNAL SQUARE  06/03/2019        61927.0
6949  JOURNAL SQUARE  06/04/2019        26989.0
6950  JOURNAL SQUARE  06/05/2019        27881.0
6951  JOURNAL SQUARE  06/06/2019        28089.0
6952  JOURNAL SQUARE  06/07/2019        10101.0
6953  JOURNAL SQUARE  06/08/2019      4444591.0
6954  JOURNAL SQUARE  06/09/2019        11585.0
6955  JOURNAL SQUARE  06/10/2019        25253.0
6956  JOURNAL SQUARE  06/11/2019        27886.0
6957  JOURNAL SQUARE  06/12/2019        28544.0
6958  JOURNAL SQUARE  06/13/2019        27346.0
6959  JOURNAL SQUARE  06/14/2019        27654.0
6960  JOURNAL SQUARE  06/15/2019        15756.0
6961  JOURNAL SQUARE  06/16/2019        10635.0
6962  JOURNAL SQUARE  06/17/2019        27171.0
6963  JOURNAL SQUARE  06/18/2019        27074.0
6964  JOURNAL SQUARE  06/19/2019        28029.0
6965  JOURNAL SQUARE  06/20/2019        27624.0
6966  JOURNAL SQUARE  06/21/2019        26503.0
6967  JOURNAL SQUARE  06/22/2019        16512.0
6968  JOURNAL SQUARE  06/23/2019        12034.0
6969  JOURNAL SQUARE  06/24/2019        27661.0
6970  JOURNAL SQUARE  06/25/2019        27130.0
6971  JOURNAL SQUARE  06/26/2019        28649.0
6972  JOURNAL SQUARE  06/27/2019        27005.0
6973  JOURNAL SQUARE  06/28/2019        28070.0

In [120]:
# (sum across all days is a reasonable way to define this)
stations = \
    (stations_daily.groupby(['STATION'])['DAILY_ENTRIES'].sum()
                   .reset_index()
                   .sort_values(by='DAILY_ENTRIES',ascending=False))

stations[stations["STATION"] == "JOURNAL SQUARE"]

STATION  DAILY_ENTRIES
260  JOURNAL SQUARE      5088833.0

In [113]:
sns.boxplot(turnstiles_daily["DAILY ENTRIES"], turnstiles_daily ["STATION"]);

KeyError: 'DAILY ENTRIES'